In [190]:
# See utils.py for the imports

from utils import *
import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_colwidth = None
pd.options.display.max_columns = None
import timeit
import nest_asyncio
nest_asyncio.apply()
#%load_ext line_profiler

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Where to save the figures (adapted this from https://github.com/ageron/handson-ml2)
PROJECT_ROOT_DIR = "."
PROJECT_ID = "stock_movement_tweet_data_wrangling"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", PROJECT_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=72):
    """ 
    resolution quality
    300 high 
    150 medium
    72 low
    """
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
plt.style.use('fivethirtyeight')

In [191]:
# imports 
# https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle

import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential 
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
stop_words = stopwords.words('english')

In [192]:
# Helper Functions

def combine_tweets_user(df):
    # convert dates to datetime and split
    datetimes = pd.to_datetime(df['created_at'])
    df['time'], df['date'] = datetimes.dt.time, datetimes.dt.date
    
    # get open and closing times
    open_t = dt.strptime('09:30:00', '%H:%M:%S').time()
    close_t = dt.strptime('16:00:00', '%H:%M:%S').time()
    
    # filter dataframes based off when tweet occurs
    before_df = df[df['time'] < open_t]
    during_df = df[((df['time'] >= open_t) & (df['time'] < close_t))]
    after_df = df[df['time'] >= close_t]
    
    # reset tweet times to next open or close time
    before_df['time'] = open_t
    during_df['time'] = close_t
    
    # if tweet occurs after close, set it to next day and set time to open
    after_df['date'] += timedelta(days=1)
    after_df['time'] = open_t

    # reassemble dataframes with updated dates/times
    df = before_df.append(during_df).append(after_df)
    
    # sort by date and time
    df['date'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
    df = df.sort_values(by='date', ascending=True)
    
    tweet = []
    combined_df = pd.DataFrame(index=sorted(list(set(df['date']))))
    combined_df['tweet'] = [[] for _ in range(len(combined_df))]
    
    # iterate over dataframe and add tokenized tweets to matching dates
    #tokenizer = TweetTokenizer()
    for i in range(len(combined_df)):
        time = combined_df.index[i]
        matching_dates = df[df['date'] == time]
        for tweet in matching_dates['tweet']:
            combined_df['tweet'].iloc[i].append(tweet.split())
            #combined_df['tweet'].iloc[i].append(df.iloc[i]['created_at'])
            #combined_df.iloc[i]['tweet'].append(df.iloc[i]['tweet'])
    # add number of tweets and username and return
    combined_df['num_tweets'] = combined_df['tweet'].apply(lambda x: len(x) )
    combined_df['username'] = df['username'].unique()[0]
    return combined_df

def combine_tweets(full_df, verbose=False):
    """
    group tweets together by next open/close date
    
    Arguments:
    full_df -- A dataframe of users tweets and created_at dates. 
    
    Returns:
    merged_df -- A dataframe of grouped user tweets as lists before the next open/close datetime
  
    A tweet is grouped via the following criteria:
    market_open = 09:30:00 EST
    market_close = 16:00:00 EST
    
    - If tweet created_at occurs before market_open on same date, it is grouped as before_market_tweets. 
    - It tweet is on or after market_open but before market_close, (i.e. during market hours) it is grouped as during_market_tweets. 
    - If tweet is on or after market_close, it is grouped as after_market_tweets.
    """
    
    users = full_df.username.unique()
    merged_df = pd.DataFrame()
    if verbose:
        max_length = 11963
        start = timeit.default_timer()
        print('')
        print('digesting dataframes....')
        print('---{:.2f} % complete -------'.format(0.0))
    for user in users:
        merged_df = merged_df.append(combine_tweets_user(full_df[full_df['username'] == user]))
        if verbose:
            print('')
            print('--{:.2f} % complete -------'.format(100*len(merged_df) / max_length))
    if verbose:
        stop = timeit.default_timer()
        print('digestion completed ---- runtime {:.2f} seconds'.format(stop - start))
    return merged_df.reset_index(drop=False).rename(columns={'index':'date'})
    
def combine_tweets_stocks(ceos_merged, stocks_full):
    new_df = pd.DataFrame()
    for user in ceos_merged['username'].unique():
        ticker = handles_tickers[user]
        tweet_df = ceos_merged[ceos_merged['username'] == user]
        stock_df = stocks_full[stocks_full['ticker'] == ticker]
        new_df = new_df.append(tweet_df.merge(stock_df, how='left', on='date')).dropna(subset=['ticker'])  
    return new_df

def fix_closed_market_tweets(test):
    test = test.dropna()
    test.reset_index(inplace=True, drop=True)
    test = test.set_index('date')
    for i in range(len(test)):
        test['tweet'].iloc[i] += " "

    i = 0
    combined=pd.DataFrame()

    while i < len(test):
        if test['price'].iloc[i] == 0:
            combined = test.iloc[i]
            j = i + 1
            while test['price'].iloc[j] == 0:
                combined += test.iloc[j]
                j += 1
            username = test.iloc[j]['username']
            price = test.iloc[j]['price']
            percent_change = test.iloc[j]['percent change']
            test.iloc[j] += combined
            test['username'].iloc[j] = username
            test['price'].iloc[j] = price
            test['percent change'].iloc[j] = percent_change
            i = j
        i += 1
    test = test[test['price'] !=0]
    return test

def organize_stocks(df): 
    """
    Combines the open/close dates and prices into a single dataframe
    """
    stock_open = pd.DataFrame()
    stock_close = pd.DataFrame()

    stock_open['date'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d 09:30:00'))
    stock_close['date'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d 16:00:00'))

    stock_open['price'] = df['open']
    stock_close['price'] = df['close']
    
    start_date_open = dt.strftime(stock_open.date.min(), '%Y-%m-%d %H:%M:%S')
    start_date_close = dt.strftime(stock_close.date.min(), '%Y-%m-%d %H:%M:%S')

    end_date_open = dt.strftime(stock_open.date.max(), '%Y-%m-%d %H:%M:%S')
    end_date_close = dt.strftime(stock_close.date.max(), '%Y-%m-%d %H:%M:%S')
    
    date_indx_open = pd.Series(pd.date_range(start_date_open, end_date_open), name='date')
    date_indx_close = pd.Series(pd.date_range(start_date_close, end_date_close), name='date')
    
    stock_open = pd.merge(date_indx_open, stock_open, how='left')
    stock_close = pd.merge(date_indx_close, stock_close, how='left')
    
    stock = pd.concat([stock_open, stock_close])
    return stock.sort_values(by='date', ascending=True)

# Load the data

In [193]:
tweets = pd.read_pickle('./data/ceos.pkl')
stock_names = ['TSLA', 'BOX', 'TWTR', 'CRM', 'SPCE', 'TMUS']
stocks = {stock:pd.read_pickle(f'data/{stock}.pkl') for stock in stock_names}
tweets_combined = combine_tweets(tweets, verbose=True)


digesting dataframes....
---0.00 % complete -------

--18.38 % complete -------

--26.44 % complete -------

--38.93 % complete -------

--54.63 % complete -------

--71.44 % complete -------

--100.00 % complete -------
digestion completed ---- runtime 84.09 seconds


In [194]:
# create a mapping between twitter handles and company stock tickers
handles = tweets['username'].unique()
tickers = stocks.keys()
handles_tickers = dict(zip(handles, tickers))
print(handles_tickers.items())
# Should be ('elonmusk', 'TSLA'), ('levie', 'BOX'), ('jack', 'TWTR'), ('Benioff', 'CRM'), ('richardbranson', 'SPCE'), ('JohnLegere', 'TMUS')

dict_items([('elonmusk', 'TSLA'), ('levie', 'BOX'), ('jack', 'TWTR'), ('Benioff', 'CRM'), ('richardbranson', 'SPCE'), ('JohnLegere', 'TMUS')])


In [195]:
print(stocks[list(stocks.keys())[0]][['Date','Open', 'Close']].head(3))

        Date       Open      Close
0 2016-08-23  44.863998  44.967999
1 2016-08-24  45.410000  44.523998
2 2016-08-25  44.622002  44.192001


In [196]:
# combine open/closed prices
stocks_org = {}
for stock in stocks:
    new_stock = stocks[stock].reset_index()
    new_stock =  stocks[stock][['Date','Open', 'Close']]
    new_stock.columns = new_stock.columns.str.lower()
    stocks_org[stock] = organize_stocks(new_stock)
    stocks_org[stock]['shifted'] = stocks_org[stock]['price'].shift() # number of lags (default is 1)
    stocks_org[stock]['change'] = stocks_org[stock]['price'].div(stocks_org[stock]['shifted'])
    stocks_org[stock]['return'] = stocks_org[stock]['change'].sub(1).mul(100)
print(stocks_org[list(stocks.keys())[0]].head(6))

                 date      price    shifted    change    return
0 2016-08-23 09:30:00  44.863998        NaN       NaN       NaN
0 2016-08-23 16:00:00  44.967999  44.863998  1.002318  0.231812
1 2016-08-24 09:30:00  45.410000  44.967999  1.009829  0.982924
1 2016-08-24 16:00:00  44.523998  45.410000  0.980489 -1.951116
2 2016-08-25 09:30:00  44.622002  44.523998  1.002201  0.220114
2 2016-08-25 16:00:00  44.192001  44.622002  0.990363 -0.963651


In [197]:
# merge into single dataframe 
stocks_full = pd.DataFrame()
for stock in stocks_org:
    stocks_org[stock]['ticker'] = stock
    stocks_full = stocks_full.append(stocks_org[stock])
stocks_full['date'] = stocks_full['date'].dt.tz_localize(tz='US/Eastern')
stocks_full = stocks_full.set_index('date')

In [198]:
tweets_combined['date'] = pd.to_datetime(tweets_combined['date'])
tweets_combined['date'] = tweets_combined['date'].dt.tz_localize(tz='US/Eastern')

In [199]:
tweets_combined

,date,tweet,num_tweets,username
0,2016-07-20 16:00:00-04:00,"[[Finishing, off, the, plan, while, listening, to, the, soundtrack, from, Gatsby., Seems, appropriate...], [Not, easy, to, convey, irony, in, a, tweet], [Except, for, the, tweet, about, large, amounts, of, crack, (actually, just, small, amounts)], [@AnnaDotBlue, busted], [@vitiral, yup], [Post, should, go, live, on, Tesla, website, around, 5pm], [California, time], [@princeghosh97, Lana]]",8,elonmusk
1,2016-07-21 09:30:00-04:00,"[[Tesla, Master, Plan,, Part, Deux, https://t.co/XQeTlsoEdE], [@danahull, It's, inspired, by, some, of, the, California, Custom, VW, combi, design, art], [@danahull, Jerome, is, driving, Tesla, Semi, &amp;, doing, a, great, job, with, his, team., At, Daimler,, he, led, their, most, successful, semi, truck, program, ever.], [Autopilot, prevents, serious, injury, or, death, of, a, pedestrian, in, NY, (owner, anecdote, confirmed, by, vehicle, logs), https://t.co/NceuqckqCK], [@elonmusk, now, 2Pac], [@katiefehren, Both, should, have, been, done, under, the, same, corporate, umbrella, from, the, beginning], [@MRamseyWSJ, Great, game., Whole, Fallout, series, is, amazing.]]",7,elonmusk
2,2016-07-22 09:30:00-04:00,"[[Looking, forward, to, the, Gigafactory, opening, party, on, Friday, next, week., The, scale, blows, me, away, every, time, I, see, it.]]",1,elonmusk
3,2016-07-23 09:30:00-04:00,"[[https://t.co/Kgkz1Z3FwJ], [@vicentes, as, long, as, I'm, alive]]",2,elonmusk
4,2016-07-25 16:00:00-04:00,"[[@ID_AA_Carmack, Maybe, as, a, future, optimization., Would, only, matter, if, it, was, extremely, big.]]",1,elonmusk
...,...,...,...,...
11958,2021-07-18 16:00:00-04:00,"[[is, it, possible, that, the, ocean, makes, the, morning, coffee, ☕️, even, better, than, it's, normal, great?, 😊, https://t.co/43jhIroufd], [Hey, New, Yorkers, –, need, an, officiant?!, https://t.co/PbBecmjgu0], [Who’s, subscribing??, https://t.co/bZr4a84P2X]]",3,JohnLegere
11959,2021-07-19 09:30:00-04:00,"[[Number, 16, is, a, movie, night, game, changer!!, https://t.co/9YJS0H383P]]",1,JohnLegere
11960,2021-07-19 16:00:00-04:00,"[[What, I’ve, learned, from, @Lifehacker, is, that, I, do, EVERYTHING, wrong!, https://t.co/xAbzXxZhqZ], [Always, love, a, good, Quesarito, https://t.co/KaiMmXEc5q], [So, I'm, about, to, go, for, a, run, on, the, beach.....am, i, the, only, one, that, doesn't, think, these, ☁️, mean, 🌧, ☔️, is, coming?, https://t.co/fMxzuOZ0nF], [Gorgeous!, Maybe, I, should, charter, a, magenta, rocket., https://t.co/jx2ZL0gX6Q]]",4,JohnLegere
11961,2021-07-20 09:30:00-04:00,"[[Call, on, John, is, POSTPONED, for, today, but, tune, in, on, Thursday,, July, 22nd, at, 12, PM, ET, on, Instagram, Stories, for, an, extra, special, Call, on, John!, What, do, you, want, to, see, me, give, away??, https://t.co/4P3hucQ3io]]",1,JohnLegere


In [200]:
combined_df = combine_tweets_stocks(tweets_combined, stocks_full)

In [201]:
combined_df = combined_df.set_index('date')
combined_price_null = combined_df[combined_df['price'].isnull()]
combined_price_not_null = combined_df[combined_df['price'].notnull()]

In [205]:
for user in combined_df['username'].unique():
    
    user_df = combined_df[combined_df['username'] == user]
    user_price_null = user_df[user_df['price'].isnull()]
    user_return_not_null = user_df[user_df['return'].notnull()]
    
    new_df = pd.DataFrame()
    
    for user in combined_df['username'].unique():
        user_df = combined_df[combined_df['username'] == user]
        #user_df = user_df.set_index('date')
        user_price_null = user_df[user_df['price'].isnull()]
        user_price_not_null = user_df[user_df['price'].notnull()]
        for i in range(len(user_price_null)):
            first_time_point = user_price_null.iloc[i].name
            next_time_point = user_return_not_null[user_return_not_null.index > first_time_point].iloc[0].name
            temp_df = user_df.loc[first_time_point : next_time_point]
            combined_tweet = []
            combined_user_df = user_price_not_null.copy()
            for j in range(len(temp_df)):
                combined_tweet += temp_df['tweet'].iloc[j]
            print(user, next_time_point)
            combined_user_df.at[next_time_point, 'tweet'] = combined_tweet
            combined_user_df['num_tweets'] = combined_user_df['tweet'].apply(lambda x: len(x))
        new_df = new_df.append(combined_user_df)

elonmusk 2016-08-30 16:00:00-04:00
elonmusk 2016-09-13 09:30:00-04:00
elonmusk 2016-09-13 09:30:00-04:00
elonmusk 2016-09-20 09:30:00-04:00
elonmusk 2016-09-20 09:30:00-04:00
elonmusk 2016-09-26 16:00:00-04:00
elonmusk 2016-10-04 16:00:00-04:00
elonmusk 2016-10-04 16:00:00-04:00
elonmusk 2016-10-04 16:00:00-04:00
elonmusk 2016-10-10 16:00:00-04:00
elonmusk 2016-10-10 16:00:00-04:00
elonmusk 2016-10-19 09:30:00-04:00
elonmusk 2016-10-26 16:00:00-04:00
elonmusk 2016-10-26 16:00:00-04:00
elonmusk 2016-11-02 09:30:00-04:00
elonmusk 2016-11-15 09:30:00-05:00
elonmusk 2016-11-15 09:30:00-05:00
elonmusk 2016-11-22 16:00:00-05:00
elonmusk 2016-12-01 09:30:00-05:00
elonmusk 2016-12-01 09:30:00-05:00
elonmusk 2016-12-21 16:00:00-05:00
elonmusk 2016-12-21 16:00:00-05:00
elonmusk 2016-12-21 16:00:00-05:00
elonmusk 2016-12-21 16:00:00-05:00
elonmusk 2016-12-21 16:00:00-05:00
elonmusk 2016-12-30 09:30:00-05:00
elonmusk 2016-12-30 09:30:00-05:00
elonmusk 2016-12-30 09:30:00-05:00
elonmusk 2017-01-06 

elonmusk 2019-02-20 09:30:00-05:00
elonmusk 2019-02-20 09:30:00-05:00
elonmusk 2019-02-25 16:00:00-05:00
elonmusk 2019-03-04 16:00:00-05:00
elonmusk 2019-03-04 16:00:00-05:00
elonmusk 2019-03-04 16:00:00-05:00
elonmusk 2019-03-12 09:30:00-04:00
elonmusk 2019-03-12 09:30:00-04:00
elonmusk 2019-03-12 09:30:00-04:00
elonmusk 2019-03-20 09:30:00-04:00
elonmusk 2019-03-20 09:30:00-04:00
elonmusk 2019-03-20 09:30:00-04:00
elonmusk 2019-03-26 09:30:00-04:00
elonmusk 2019-04-01 16:00:00-04:00
elonmusk 2019-04-01 16:00:00-04:00
elonmusk 2019-04-01 16:00:00-04:00
elonmusk 2019-04-01 16:00:00-04:00
elonmusk 2019-04-08 16:00:00-04:00
elonmusk 2019-04-08 16:00:00-04:00
elonmusk 2019-04-08 16:00:00-04:00
elonmusk 2019-04-08 16:00:00-04:00
elonmusk 2019-04-16 09:30:00-04:00
elonmusk 2019-04-16 09:30:00-04:00
elonmusk 2019-04-16 09:30:00-04:00
elonmusk 2019-04-16 09:30:00-04:00
elonmusk 2019-04-22 16:00:00-04:00
elonmusk 2019-04-22 16:00:00-04:00
elonmusk 2019-04-22 16:00:00-04:00
elonmusk 2019-04-22 

elonmusk 2020-11-16 16:00:00-05:00
elonmusk 2020-11-16 16:00:00-05:00
elonmusk 2020-11-24 09:30:00-05:00
elonmusk 2020-11-24 09:30:00-05:00
elonmusk 2020-11-24 09:30:00-05:00
elonmusk 2020-11-30 16:00:00-05:00
elonmusk 2020-11-30 16:00:00-05:00
elonmusk 2020-11-30 16:00:00-05:00
elonmusk 2020-11-30 16:00:00-05:00
elonmusk 2020-11-30 16:00:00-05:00
elonmusk 2020-12-07 16:00:00-05:00
elonmusk 2020-12-07 16:00:00-05:00
elonmusk 2020-12-14 16:00:00-05:00
elonmusk 2020-12-14 16:00:00-05:00
elonmusk 2020-12-14 16:00:00-05:00
elonmusk 2020-12-21 16:00:00-05:00
elonmusk 2020-12-21 16:00:00-05:00
elonmusk 2020-12-28 16:00:00-05:00
elonmusk 2020-12-28 16:00:00-05:00
elonmusk 2020-12-28 16:00:00-05:00
elonmusk 2020-12-28 16:00:00-05:00
elonmusk 2020-12-28 16:00:00-05:00
elonmusk 2020-12-28 16:00:00-05:00
elonmusk 2021-01-04 16:00:00-05:00
elonmusk 2021-01-04 16:00:00-05:00
elonmusk 2021-01-04 16:00:00-05:00
elonmusk 2021-01-04 16:00:00-05:00
elonmusk 2021-01-11 16:00:00-05:00
elonmusk 2021-01-11 

ValueError: Must have equal len keys and value when setting with an iterable

In [210]:
test = combined_df[combined_df['username'] == 'levie']
test.loc['2016-09']

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-09-01 16:00:00-04:00,"[[Unlike, essentially, every, other, kind, of, challenge, a, startup, runs, into,, you, probably, don't, hear, ""it's, not, rocket, science"", much, at, SpaceX.]]",1,levie,13.98,13.25,1.055094,5.509431,BOX
2016-09-02 16:00:00-04:00,"[[Hopefully, it's, clear, by, now, that, promising, taco, trucks, on, every, corner, would,, in, fact,, be, the, most, electable, platform, to, run, on.]]",1,levie,14.14,14.00,1.010000,1.000002,BOX
2016-09-03 09:30:00-04:00,"[[BoxWorks, is, now, 5, days, away!, Grab, a, free, expo, pass, to, join, us, for, the, keynotes, here:, https://t.co/4Pgi6yBzXa, https://t.co/4CtLndUeUz]]",1,levie,NaN,14.14,NaN,NaN,BOX
2016-09-04 09:30:00-04:00,"[[Building, an, enterprise, software, company, is, 95%, technology, and, 5%, channeling, your, inner, Taylor, Swift., https://t.co/vrXrXt55Rr]]",1,levie,NaN,NaN,NaN,NaN,BOX
2016-09-05 09:30:00-04:00,"[[""Oh, sure,, this, seems, like, a, totally, safe, and, normal, Airbnb, to, stay, in."", https://t.co/vfulGB7ftx]]",1,levie,NaN,NaN,NaN,NaN,BOX
2016-09-06 09:30:00-04:00,"[[7,000, people, registered, for, BoxWorks, this, week!, If, you, can't, make, it, (why!?, 😢😟😞😗), we'll, be, streaming, live, here:, https://t.co/EXn9NZAbk0]]",1,levie,14.14,NaN,NaN,NaN,BOX
2016-09-07 16:00:00-04:00,"[[Steps, for, a, great, morning:, 1., Load, https://t.co/OefY681YEc, 2., Load, https://t.co/MFBvDV5Hnj, 3., Experience, all, your, tech, wishes, come, true]]",1,levie,14.52,14.32,1.013967,1.396653,BOX
2016-09-08 09:30:00-04:00,"[[@PeggyJ, are, you, calling, my, fashion, out, of, date, 😜?, Of, course, it's, the, latest, version, 😝!, Looking, forward, to, our, race!]]",1,levie,14.50,14.52,0.998623,-0.137744,BOX
2016-09-11 09:30:00-04:00,"[[Me:, We, had, thousands, of, people, at, our, conference!, 98, y/o, grandpa:, Can't, you, just, do, it, electronically?, Touché.]]",1,levie,NaN,NaN,NaN,NaN,BOX


In [180]:
%%time
new_df = pd.DataFrame()
for user in combined_df['username'].unique():
    user_df = combined_df[combined_df['username'] == user]
    #user_df = user_df.set_index('date')
    user_price_null = user_df[user_df['price'].isnull()]
    user_price_not_null = user_df[user_df['price'].notnull()]
    for i in range(len(user_price_null)):
        first_time_point = user_price_null.iloc[i].name
        # find where next timestamp would be
        next_time_point = user_price_not_null[user_price_not_null.index > first_time_point].iloc[0].name
        temp_df = user_df.loc[first_time_point : next_time_point]
        combined_tweet = []
        combined_user_df = user_price_not_null.copy()
        for j in range(len(temp_df)):
            combined_tweet += temp_df['tweet'].iloc[j]
        combined_user_df.at[next_time_point, 'tweet'] = combined_tweet
        combined_user_df['num_tweets'] = combined_user_df['tweet'].apply(lambda x: len(x))
    new_df = new_df.append(combined_user_df)

CPU times: user 10.9 s, sys: 258 ms, total: 11.1 s
Wall time: 12.4 s


In [187]:
new_df['dayofweek'] = new_df.index.day_name()

In [188]:
new_df

,tweet,num_tweets,username,price,shifted,change,return,ticker,dayofweek
date,,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q&amp;A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today]]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA,Tuesday
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA,Tuesday
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed,, according, to, HK, regulations., Happy, to, do, it, if, regs, change.], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details., Will, publish, on, Tesla, website, later, today.], [Major, improvements, to, Autopilot, coming, with, V8.0, and, 8.1, software, (std, OTA, update), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh]]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA,Wednesday
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation., Originated, around, upper, stage, oxygen, tank., Cause, still, unknown., More, soon.]]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA,Thursday
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes., This, seems, instant, from, a, human, perspective,, but, it, really, a, fast, fire,, not, an, explosion., Dragon, would, have, been, fine.]]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA,Friday
...,...,...,...,...,...,...,...,...,...
2021-07-16 16:00:00-04:00,"[[BUT, WHERE, IS, THE, SOLID, PINK, HEART?!!, https://t.co/wF8LWtG6dx], [Trust, science!, It’s, important, to, continue, staying, safe,, the, COVID-19, pandemic, isn’t, over, yet!, https://t.co/HBhDZCbvrk]]",2,JohnLegere,149.410004,148.089996,1.008914,0.891355,TMUS,Friday
2021-07-19 09:30:00-04:00,"[[is, it, possible, that, the, ocean, makes, the, morning, coffee, ☕️, even, better, than, it's, normal, great?, 😊, https://t.co/43jhIroufd], [Hey, New, Yorkers, –, need, an, officiant?!, https://t.co/PbBecmjgu0], [Who’s, subscribing??, https://t.co/bZr4a84P2X], [Number, 16, is, a, movie, night, game, changer!!, https://t.co/9YJS0H383P]]",4,JohnLegere,147.500000,NaN,NaN,NaN,TMUS,Monday
2021-07-19 16:00:00-04:00,"[[What, I’ve, learned, from, @Lifehacker, is, that, I, do, EVERYTHING, wrong!, https://t.co/xAbzXxZhqZ], [Always, love, a, good, Quesarito, https://t.co/KaiMmXEc5q], [So, I'm, about, to, go, for, a, run, on, the, beach.....am, i, the, only, one, that, doesn't, think, these, ☁️, mean, 🌧, ☔️, is, coming?, https://t.co/fMxzuOZ0nF], [Gorgeous!, Maybe, I, should, charter, a, magenta, rocket., https://t.co/jx2ZL0gX6Q]]",4,JohnLegere,144.610001,147.500000,0.980407,-1.959322,TMUS,Monday


In [177]:
# collect first null point
for i in range(len(user_price_null)):
    first_time_point = user_price_null.iloc[i].name
    # find where next timestamp would be
    next_time_point = user_price_not_null[user_price_not_null.index > first_time_point].iloc[0].name
    temp_df = user_df.loc[first_time_point : next_time_point]
    combined_tweet = []
    combined_user_df = user_price_not_null.copy()
    for j in range(len(temp_df)):
        combined_tweet += temp_df['tweet'].iloc[j]
    combined_user_df.at[next_time_point, 'tweet'] = combined_tweet
    
    combined_user_df['num_tweets'] = combined_user_df['tweet'].apply(lambda x: len(x))

In [178]:
combined_user_df

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q&amp;A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today]]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress., Maybe, something, to, announce, in, a, few, months., Have, played, all, prior, Deus, Ex., Not, this, one, yet.], [Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",3,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed,, according, to, HK, regulations., Happy, to, do, it, if, regs, change.], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details., Will, publish, on, Tesla, website, later, today.], [Major, improvements, to, Autopilot, coming, with, V8.0, and, 8.1, software, (std, OTA, update), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh]]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation., Originated, around, upper, stage, oxygen, tank., Cause, still, unknown., More, soon.]]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes., This, seems, instant, from, a, human, perspective,, but, it, really, a, fast, fire,, not, an, explosion., Dragon, would, have, been, fine.]]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-16 09:30:00-04:00,"[[@AaronS5_, @FrenchieEAP, @karpathy, Yes], [@AustinTeslaClub, @SpaceX, @austinbarnard45, @PPathole, @TeslaOwnersEBay, @bluemoondance74, @teslaownersSV, @JohnnaCrider1, @TeslaNY, Absolutely!], [@FrenchieEAP, @karpathy, FSD, beta, 9, is, using, the, pure, vision, production, code, for, highway, driving., Beta, 10, hopefully, (Beta, 11, definitely), will, use, one, stack, to, rule, them, all, –, city, streets,, highway, &amp;, complex, parking, lots.], [@Erdayastronaut, @SpaceX, Probably, Monday]]",4,elonmusk,654.679993,650.599976,1.006271,0.627116,TSLA
2021-07-16 16:00:00-04:00,"[[@TeslaNY, Do, you, even, press?], [@Teslarati, Improving, permit, approval, speed, &amp;, lowering, permit, costs, for, solar, would, make, a, big, difference], [@RationalEtienne, @etherkragg, Those, are, major, factors], [@enn_nafnlaus, @jpr007, @TesLatino, @klwtts, Indeed,, but, again, like, a, parking, lot,, a, battery, having, big, “roads”, tends, to, decrease, number, of, “parking, spaces”, (ie, stores, less, energy)]]",4,elonmusk,644.219971,654.679993,0.984023,-1.597731,TSLA
2021-07-19 09:30:00-04:00,"[[@thePiggsBoson, Problem, 1st,, theory, 2nd, is, for, sure, way, to, go,, as, it, establishes, relevance,, thus, improving, memory, retention], [@DragTimes, @Tesla, Nice], [@grimnut, @Tesla, @WholeMarsBlog, @DirtyTesla, Haha], [@WholeMarsBlog, You, don’t, even, need, to, touch, the, shifter, in, new, S., Auto, detect, direction, will, come, as, an, optional, setting, to, all, cars, with, FSD.]]",4,elonmusk,629.890015,NaN,NaN,NaN,TSLA


In [162]:
next_time_point

Timestamp('2016-08-30 16:00:00-0400', tz='US/Eastern')

In [167]:
user_df.loc[first_time_point: next_time_point]

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-28 09:30:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress., Maybe, something, to, announce, in, a, few, months., Have, played, all, prior, Deus, Ex., Not, this, one, yet.]]",2,elonmusk,NaN,NaN,NaN,NaN,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",1,elonmusk,42.268002,43.222,0.977928,-2.207206,TSLA


In [169]:
temp_df = user_df.loc[first_time_point: next_time_point]
combined_tweet = []
for i in range(len(temp_df)):
    combined_tweet += temp_df['tweet'].iloc[i]
user_price_not_null.at[next_time_point, 'tweet'] = combined_tweet


In [170]:
user_price_not_null

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q&amp;A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today]]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress., Maybe, something, to, announce, in, a, few, months., Have, played, all, prior, Deus, Ex., Not, this, one, yet.], [Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed,, according, to, HK, regulations., Happy, to, do, it, if, regs, change.], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details., Will, publish, on, Tesla, website, later, today.], [Major, improvements, to, Autopilot, coming, with, V8.0, and, 8.1, software, (std, OTA, update), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh]]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation., Originated, around, upper, stage, oxygen, tank., Cause, still, unknown., More, soon.]]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes., This, seems, instant, from, a, human, perspective,, but, it, really, a, fast, fire,, not, an, explosion., Dragon, would, have, been, fine.]]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-16 09:30:00-04:00,"[[@AaronS5_, @FrenchieEAP, @karpathy, Yes], [@AustinTeslaClub, @SpaceX, @austinbarnard45, @PPathole, @TeslaOwnersEBay, @bluemoondance74, @teslaownersSV, @JohnnaCrider1, @TeslaNY, Absolutely!], [@FrenchieEAP, @karpathy, FSD, beta, 9, is, using, the, pure, vision, production, code, for, highway, driving., Beta, 10, hopefully, (Beta, 11, definitely), will, use, one, stack, to, rule, them, all, –, city, streets,, highway, &amp;, complex, parking, lots.], [@Erdayastronaut, @SpaceX, Probably, Monday]]",4,elonmusk,654.679993,650.599976,1.006271,0.627116,TSLA
2021-07-16 16:00:00-04:00,"[[@TeslaNY, Do, you, even, press?], [@Teslarati, Improving, permit, approval, speed, &amp;, lowering, permit, costs, for, solar, would, make, a, big, difference], [@RationalEtienne, @etherkragg, Those, are, major, factors], [@enn_nafnlaus, @jpr007, @TesLatino, @klwtts, Indeed,, but, again, like, a, parking, lot,, a, battery, having, big, “roads”, tends, to, decrease, number, of, “parking, spaces”, (ie, stores, less, energy)]]",4,elonmusk,644.219971,654.679993,0.984023,-1.597731,TSLA
2021-07-19 09:30:00-04:00,"[[@DragTimes, @Tesla, Nice], [@grimnut, @Tesla, @WholeMarsBlog, @DirtyTesla, Haha], [@WholeMarsBlog, You, don’t, even, need, to, touch, the, shifter, in, new, S., Auto, detect, direction, will, come, as, an, optional, setting, to, all, cars, with, FSD.]]",3,elonmusk,629.890015,NaN,NaN,NaN,TSLA


In [148]:
user_price_not_null[user_price_not_null.index == next_time_point]['tweet'] = combined_tweet

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",1,elonmusk,42.268002,43.222,0.977928,-2.207206,TSLA


In [144]:
user_price_not_null

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q&amp;A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today]]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed,, according, to, HK, regulations., Happy, to, do, it, if, regs, change.], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details., Will, publish, on, Tesla, website, later, today.], [Major, improvements, to, Autopilot, coming, with, V8.0, and, 8.1, software, (std, OTA, update), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh]]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation., Originated, around, upper, stage, oxygen, tank., Cause, still, unknown., More, soon.]]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes., This, seems, instant, from, a, human, perspective,, but, it, really, a, fast, fire,, not, an, explosion., Dragon, would, have, been, fine.]]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-16 09:30:00-04:00,"[[@AaronS5_, @FrenchieEAP, @karpathy, Yes], [@AustinTeslaClub, @SpaceX, @austinbarnard45, @PPathole, @TeslaOwnersEBay, @bluemoondance74, @teslaownersSV, @JohnnaCrider1, @TeslaNY, Absolutely!], [@FrenchieEAP, @karpathy, FSD, beta, 9, is, using, the, pure, vision, production, code, for, highway, driving., Beta, 10, hopefully, (Beta, 11, definitely), will, use, one, stack, to, rule, them, all, –, city, streets,, highway, &amp;, complex, parking, lots.], [@Erdayastronaut, @SpaceX, Probably, Monday]]",4,elonmusk,654.679993,650.599976,1.006271,0.627116,TSLA
2021-07-16 16:00:00-04:00,"[[@TeslaNY, Do, you, even, press?], [@Teslarati, Improving, permit, approval, speed, &amp;, lowering, permit, costs, for, solar, would, make, a, big, difference], [@RationalEtienne, @etherkragg, Those, are, major, factors], [@enn_nafnlaus, @jpr007, @TesLatino, @klwtts, Indeed,, but, again, like, a, parking, lot,, a, battery, having, big, “roads”, tends, to, decrease, number, of, “parking, spaces”, (ie, stores, less, energy)]]",4,elonmusk,644.219971,654.679993,0.984023,-1.597731,TSLA
2021-07-19 09:30:00-04:00,"[[@DragTimes, @Tesla, Nice], [@grimnut, @Tesla, @WholeMarsBlog, @DirtyTesla, Haha], [@WholeMarsBlog, You, don’t, even, need, to, touch, the, shifter, in, new, S., Auto, detect, direction, will, come, as, an, optional, setting, to, all, cars, with, FSD.]]",3,elonmusk,629.890015,NaN,NaN,NaN,TSLA


In [123]:
combined_tweet

[['@Kotaku', 'one', 'of', 'my', 'favorite', 'games', 'as', 'a', 'kid'],
 ['@BelovedRevol',
  'Making',
  'progress.',
  'Maybe',
  'something',
  'to',
  'announce',
  'in',
  'a',
  'few',
  'months.',
  'Have',
  'played',
  'all',
  'prior',
  'Deus',
  'Ex.',
  'Not',
  'this',
  'one',
  'yet.'],
 ['Thanks',
  'for',
  'the',
  'longstanding',
  'faith',
  'in',
  'SpaceX.',
  'We',
  'very',
  'much',
  'look',
  'forward',
  'to',
  'doing',
  'this',
  'milestone',
  'flight',
  'with',
  'you.',
  'https://t.co/U2UFez0OhY']]

In [124]:
temp_df.iloc[-1]['tweet'] = combined_tweet

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-28 09:30:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress., Maybe, something, to, announce, in, a, few, months., Have, played, all, prior, Deus, Ex., Not, this, one, yet.]]",2,elonmusk,NaN,NaN,NaN,NaN,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",1,elonmusk,42.268002,43.222,0.977928,-2.207206,TSLA


In [103]:
user_price_not_null[user_price_not_null.index > first_point.name].iloc[0]

tweet         [[Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]
num_tweets                                                                                                                                                                 1
username                                                                                                                                                            elonmusk
price                                                                                                                                                              42.268002
shifted                                                                                                                                                               43.222
change                                                                                                                                 

In [98]:
user_price_not_null

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q&amp;A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today]]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed,, according, to, HK, regulations., Happy, to, do, it, if, regs, change.], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details., Will, publish, on, Tesla, website, later, today.], [Major, improvements, to, Autopilot, coming, with, V8.0, and, 8.1, software, (std, OTA, update), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh]]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation., Originated, around, upper, stage, oxygen, tank., Cause, still, unknown., More, soon.]]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes., This, seems, instant, from, a, human, perspective,, but, it, really, a, fast, fire,, not, an, explosion., Dragon, would, have, been, fine.]]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-16 09:30:00-04:00,"[[@AaronS5_, @FrenchieEAP, @karpathy, Yes], [@AustinTeslaClub, @SpaceX, @austinbarnard45, @PPathole, @TeslaOwnersEBay, @bluemoondance74, @teslaownersSV, @JohnnaCrider1, @TeslaNY, Absolutely!], [@FrenchieEAP, @karpathy, FSD, beta, 9, is, using, the, pure, vision, production, code, for, highway, driving., Beta, 10, hopefully, (Beta, 11, definitely), will, use, one, stack, to, rule, them, all, –, city, streets,, highway, &amp;, complex, parking, lots.], [@Erdayastronaut, @SpaceX, Probably, Monday]]",4,elonmusk,654.679993,650.599976,1.006271,0.627116,TSLA
2021-07-16 16:00:00-04:00,"[[@TeslaNY, Do, you, even, press?], [@Teslarati, Improving, permit, approval, speed, &amp;, lowering, permit, costs, for, solar, would, make, a, big, difference], [@RationalEtienne, @etherkragg, Those, are, major, factors], [@enn_nafnlaus, @jpr007, @TesLatino, @klwtts, Indeed,, but, again, like, a, parking, lot,, a, battery, having, big, “roads”, tends, to, decrease, number, of, “parking, spaces”, (ie, stores, less, energy)]]",4,elonmusk,644.219971,654.679993,0.984023,-1.597731,TSLA
2021-07-19 09:30:00-04:00,"[[@DragTimes, @Tesla, Nice], [@grimnut, @Tesla, @WholeMarsBlog, @DirtyTesla, Haha], [@WholeMarsBlog, You, don’t, even, need, to, touch, the, shifter, in, new, S., Auto, detect, direction, will, come, as, an, optional, setting, to, all, cars, with, FSD.]]",3,elonmusk,629.890015,NaN,NaN,NaN,TSLA


In [90]:
combined_price_not_null

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q&amp;A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today]]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed,, according, to, HK, regulations., Happy, to, do, it, if, regs, change.], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details., Will, publish, on, Tesla, website, later, today.], [Major, improvements, to, Autopilot, coming, with, V8.0, and, 8.1, software, (std, OTA, update), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh]]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation., Originated, around, upper, stage, oxygen, tank., Cause, still, unknown., More, soon.]]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes., This, seems, instant, from, a, human, perspective,, but, it, really, a, fast, fire,, not, an, explosion., Dragon, would, have, been, fine.]]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-16 16:00:00-04:00,"[[BUT, WHERE, IS, THE, SOLID, PINK, HEART?!!, https://t.co/wF8LWtG6dx], [Trust, science!, It’s, important, to, continue, staying, safe,, the, COVID-19, pandemic, isn’t, over, yet!, https://t.co/HBhDZCbvrk]]",2,JohnLegere,149.410004,148.089996,1.008914,0.891355,TMUS
2021-07-19 09:30:00-04:00,"[[Number, 16, is, a, movie, night, game, changer!!, https://t.co/9YJS0H383P]]",1,JohnLegere,147.500000,NaN,NaN,NaN,TMUS
2021-07-19 16:00:00-04:00,"[[What, I’ve, learned, from, @Lifehacker, is, that, I, do, EVERYTHING, wrong!, https://t.co/xAbzXxZhqZ], [Always, love, a, good, Quesarito, https://t.co/KaiMmXEc5q], [So, I'm, about, to, go, for, a, run, on, the, beach.....am, i, the, only, one, that, doesn't, think, these, ☁️, mean, 🌧, ☔️, is, coming?, https://t.co/fMxzuOZ0nF], [Gorgeous!, Maybe, I, should, charter, a, magenta, rocket., https://t.co/jx2ZL0gX6Q]]",4,JohnLegere,144.610001,147.500000,0.980407,-1.959322,TMUS


In [88]:
combined_return_null = combined_price_not_null[combined_price_not_null['return'].isnull()]

In [89]:
combined_return_null

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-09-26 09:30:00-04:00,"[[SpaceX, propulsion, just, achieved, first, firing, of, the, Raptor, interplanetary, transport, engine, https://t.co/vRleyJvBkx], [Mach, diamonds, https://t.co/TCX7ZGFnN0], [Production, Raptor, goal, is, specific, impulse, of, 382, seconds, and, thrust, of, 3, MN, (~310, metric, tons), at, 300, bar], [Chamber, pressure, is, almost, 3X, Merlin,, so, engine, is, about, the, same, size, for, a, given, area, ratio], [@rocketrepreneur, 382s, is, with, a, 150, area, ratio, vacuum, (or, Mars, ambient, pressure), nozzle., Will, go, over, specs, for, both, versions, on, Tues.], [@williamwinters, @rocketrepreneur, pretty, close], [@DavidKYoon, yes]]",7,elonmusk,41.299999,NaN,NaN,NaN,TSLA
2016-10-03 09:30:00-04:00,"[[@kakurady, @xenophrenia, Exactly., Those, are, the, best, ones.]]",1,elonmusk,42.459999,NaN,NaN,NaN,TSLA
2016-10-10 09:30:00-04:00,"[[Translating, what, actually, happened, in, the, debates..., https://t.co/WW0om1rsC8], [@ithrow88, @Walpoleboy6_RBX, impact, velocity, and, center, of, mass, of, colliding, object, make, a, big, difference]]",2,elonmusk,40.270000,NaN,NaN,NaN,TSLA
2016-10-17 09:30:00-04:00,"[[Moving, the, Tesla, announcement, to, Wednesday., Needs, a, few, more, days, of, refinement.], [@markpinc, @TeslaMotors, thanks!]]",2,elonmusk,39.410000,NaN,NaN,NaN,TSLA
2016-10-24 09:30:00-04:00,"[[@soswow, yeah,, great, questions]]",1,elonmusk,40.200001,NaN,NaN,NaN,TSLA
...,...,...,...,...,...,...,...,...
2021-06-07 09:30:00-04:00,"[[Always, love, freezer-friendly, foods, when, I, make, too, much, in, my, slow, cooker!, https://t.co/F19NciGvw1]]",1,JohnLegere,143.910004,NaN,NaN,NaN,TMUS
2021-06-14 09:30:00-04:00,"[[An, organized, kitchen, makes, me, a, happy, chef!, https://t.co/9Z8zOIKknM]]",1,JohnLegere,144.990005,NaN,NaN,NaN,TMUS
2021-06-21 09:30:00-04:00,"[[Number, 11, had, me, LOLing!, https://t.co/AfMQaLivws]]",1,JohnLegere,145.970001,NaN,NaN,NaN,TMUS


In [28]:
test = combined_df.copy()
test

,date,tweet,num_tweets,username,price,shifted,change,return,ticker
22,2016-08-23 16:00:00-04:00,"[[Journalist, Q, &, A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today], ]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
23,2016-08-28 09:30:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress, ., Maybe, something, to, announce, in, a, few, months, ., Have, played, all, prior, Deus, Ex, ., Not, this, one, yet, .]]",2,elonmusk,NaN,NaN,NaN,NaN,TSLA
24,2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX, ., We, very, much, look, forward, to, doing, this, milestone, flight, with, you, ., https://t.co/U2UFez0OhY], ]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
25,2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed, ,, according, to, HK, regulations, ., Happy, to, do, it, if, regs, change, .], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details, ., Will, publish, on, Tesla, website, later, today, .], [Major, improvements, to, Autopilot, coming, with, V8, ., 0, and, 8.1, software, (, std, OTA, update, ), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh], ]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
26,2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation, ., Originated, around, upper, stage, oxygen, tank, ., Cause, still, unknown, ., More, soon, .], ]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
...,...,...,...,...,...,...,...,...,...
3412,2021-07-18 16:00:00-04:00,"[[is, it, possible, that, the, ocean, makes, the, morning, coffee, ☕, ️, even, better, than, it's, normal, great, ?, 😊, https://t.co/43jhIroufd], [Hey, New, Yorkers, –, need, an, officiant, ?, !, https://t.co/PbBecmjgu0], [Who, ’, s, subscribing, ?, ?, https://t.co/bZr4a84P2X]]",3,JohnLegere,NaN,NaN,NaN,NaN,TMUS
3413,2021-07-19 09:30:00-04:00,"[[Number, 16, is, a, movie, night, game, changer, !, !, https://t.co/9YJS0H383P]]",1,JohnLegere,147.500000,NaN,NaN,NaN,TMUS
3414,2021-07-19 16:00:00-04:00,"[[What, I, ’, ve, learned, from, @Lifehacker, is, that, I, do, EVERYTHING, wrong, !, https://t.co/xAbzXxZhqZ], [Always, love, a, good, Quesarito, https://t.co/KaiMmXEc5q], [So, I'm, about, to, go, for, a, run, on, the, beach, ..., am, i, the, only, one, that, doesn't, think, these, ☁, ️, mean, 🌧, ☔, ️, is, coming, ?, https://t.co/fMxzuOZ0nF], [Gorgeous, !, Maybe, I, should, charter, a, magenta, rocket, ., https://t.co/jx2ZL0gX6Q], ]",4,JohnLegere,144.610001,147.500000,0.980407,-1.959322,TMUS
3415,2021-07-20 09:30:00-04:00,"[[Call, on, John, is, POSTPONED, for, today, but, tune, in, on, Thursday, ,, July, 22nd, at, 12, PM, ET, on, Instagram, Stories, for, an, extra, special, Call, on, John, !, What, do, you, want, to, see, me, give, away, ?, ?, https://t.co/4P3hucQ3io], ]",1,JohnLegere,144.789993,144.610001,1.001245,0.124468,TMUS


In [35]:
test.set_index('date')

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q, &, A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today], ]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-28 09:30:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress, ., Maybe, something, to, announce, in, a, few, months, ., Have, played, all, prior, Deus, Ex, ., Not, this, one, yet, .]]",2,elonmusk,NaN,NaN,NaN,NaN,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX, ., We, very, much, look, forward, to, doing, this, milestone, flight, with, you, ., https://t.co/U2UFez0OhY], ]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed, ,, according, to, HK, regulations, ., Happy, to, do, it, if, regs, change, .], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details, ., Will, publish, on, Tesla, website, later, today, .], [Major, improvements, to, Autopilot, coming, with, V8, ., 0, and, 8.1, software, (, std, OTA, update, ), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh], ]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation, ., Originated, around, upper, stage, oxygen, tank, ., Cause, still, unknown, ., More, soon, .], ]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
...,...,...,...,...,...,...,...,...
2021-07-18 16:00:00-04:00,"[[is, it, possible, that, the, ocean, makes, the, morning, coffee, ☕, ️, even, better, than, it's, normal, great, ?, 😊, https://t.co/43jhIroufd], [Hey, New, Yorkers, –, need, an, officiant, ?, !, https://t.co/PbBecmjgu0], [Who, ’, s, subscribing, ?, ?, https://t.co/bZr4a84P2X]]",3,JohnLegere,NaN,NaN,NaN,NaN,TMUS
2021-07-19 09:30:00-04:00,"[[Number, 16, is, a, movie, night, game, changer, !, !, https://t.co/9YJS0H383P]]",1,JohnLegere,147.500000,NaN,NaN,NaN,TMUS
2021-07-19 16:00:00-04:00,"[[What, I, ’, ve, learned, from, @Lifehacker, is, that, I, do, EVERYTHING, wrong, !, https://t.co/xAbzXxZhqZ], [Always, love, a, good, Quesarito, https://t.co/KaiMmXEc5q], [So, I'm, about, to, go, for, a, run, on, the, beach, ..., am, i, the, only, one, that, doesn't, think, these, ☁, ️, mean, 🌧, ☔, ️, is, coming, ?, https://t.co/fMxzuOZ0nF], [Gorgeous, !, Maybe, I, should, charter, a, magenta, rocket, ., https://t.co/jx2ZL0gX6Q], ]",4,JohnLegere,144.610001,147.500000,0.980407,-1.959322,TMUS


In [49]:
tesla = test[test['ticker'] == 'TSLA'].set_index('date')
tesla_null = tesla[tesla['price'].isnull()]
tesla_not_null = tesla[tesla['price'].notnull()]

In [54]:
tesla.loc['2016-08-23 16:00:00-04:00']['tweet']

[['Journalist',
  'Q',
  '&',
  'A',
  'for',
  '30',
  'mins',
  'and',
  'embargo',
  'ends',
  'at',
  '12:30'],
 ['Tesla',
  'product',
  'announcement',
  'at',
  'noon',
  'California',
  'time',
  'today'],
 ' ']

In [51]:
tesla_null

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-28 09:30:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress, ., Maybe, something, to, announce, in, a, few, months, ., Have, played, all, prior, Deus, Ex, ., Not, this, one, yet, .]]",2,elonmusk,NaN,NaN,NaN,NaN,TSLA
2016-09-10 09:30:00-04:00,"[[Thoughtful, Op-ed, in, Space, News, much, appreciated, https://t.co/CJq5g3NIEK]]",1,elonmusk,NaN,38.894001,NaN,NaN,TSLA
2016-09-10 16:00:00-04:00,"[[Will, do, some, press, Q, &, A, on, Autopilot, post, at, 11am, PDT, tmrw, and, then, publish, at, noon, ., Sorry, about, delay, ., Unusually, difficult, couple, of, weeks, .]]",1,elonmusk,NaN,NaN,NaN,NaN,TSLA
2016-09-17 09:30:00-04:00,"[[@kzmakino, sounds, about, right], [Turns, out, MCT, can, go, well, beyond, Mars, ,, so, will, need, a, new, name, …], [I, love, the, suggestions, !], [Maybe, Ultimate, Spaceship, ,, Version, 2, ?, Mostly, because, it, is, not, the, ultimate, and, there, isn't, a, version, 1, .], [@TheMikeBartley, yeah, !], [@LarryMasters778, That, is, my, favorite, fictional, spaceship, ., Powered, by, improbability, !], [@andyzalk, sounds, like, the, right, name, for, the, rocket, booster], [@asunshinegreene, yes], [@JonasPolsky, @schellgames, that's, a, great, one]]",9,elonmusk,NaN,41.080002,NaN,NaN,TSLA
2016-09-18 09:30:00-04:00,"[[@thescalex, @SpaceX, @IAC2016, @YouTube, its, it, is], [Preview, of, the, @SpaceX, interplanetary, transport, system, at, @IAC2016, https://t.co/Rz4XmeAoRw]]",2,elonmusk,NaN,NaN,NaN,NaN,TSLA
...,...,...,...,...,...,...,...,...
2021-07-11 16:00:00-04:00,"[[@richardbranson, @virgingalactic, Congratulations, ,, beautiful, flight, !], [@wapodavenport, Spaceport, design, is, cool], [@vincent13031925, Final, decision, made, earlier, this, week, on, booster, engine, count, ., Will, be, 33, at, ~, 230, (, half, million, lbs, ), sea-level, thrust, ., All, engines, on, booster, are, same, ,, apart, from, deleting, gimbal, &, thrust, vector, actuators, for, outer, 20, .]]",3,elonmusk,NaN,NaN,NaN,NaN,TSLA
2021-07-17 09:30:00-04:00,"[[@fael097, Pure, coincidence, !], [@ValaAfshar, Even, smaller, to, a, @neuralink, chip], [@ErcXspace, @NASASpaceflight, @SpaceX, Very, accurate, !], [@JeffTutorials, @TonyTesla4Life, @WholeMarsBlog, Yes], [@techAU, Roughly], [@Charly923407591, @techAU, Haha], [@ErcXspace, @NASASpaceflight, @SpaceX, Some, of, these, design, trades, are, still, open, ,, but, will, be, resolved, soon], [@facebookai, To, date, ,, AI, chatbots, have, had, a, rather, short, MtH, (, meantime, to, Hitler, ), score, ., Tay, was, ~, 16, hours, ., https://t.co/FnWMXgpZji], [@TonyTesla4Life, @WholeMarsBlog, Wide, beta, maybe, with, FSD, rev, 10, ,, definitely, with, rev, 11]]",9,elonmusk,NaN,644.219971,NaN,NaN,TSLA
2021-07-17 16:00:00-04:00,"[[@SamTwits, Nice], [https://t.co/d4ZOSKZESP], [Tap, on, the, screen, https://t.co/YPyyj8V8DF], [@engineers_feed, A, classic], [@lexfridman, “, All, your, basis, points, are, belong, to, us, ”, -, fiat, issuers], [@AshleyIllusion1, @lexfridman, Lil, X, is, hodling, his, Doge, like, a, champ, ., Literally, never, said, the, word, “, sell, ”, even, once, !], [@riorahardi618, True], [@billycrammer, @Tesla, Cool, !], [@engineers_feed, There, ’, s, a, corner, case, where, brick, density, is, same, density, as, water, ,, reaching, bottom, due, to, momentum], [Fred, Astaire, is, incredible, ., Worth, watching, his, movies, ., One, of, a, kind, .], [@ValaAfshar, Haha], [@TrungTPhan, Now, ,, he, can, bench, press, a, rhino]]",12,elonmusk,NaN,NaN,NaN,NaN,TSLA


In [52]:
tesla_not_null

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q, &, A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today], ]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX, ., We, very, much, look, forward, to, doing, this, milestone, flight, with, you, ., https://t.co/U2UFez0OhY], ]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed, ,, according, to, HK, regulations, ., Happy, to, do, it, if, regs, change, .], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details, ., Will, publish, on, Tesla, website, later, today, .], [Major, improvements, to, Autopilot, coming, with, V8, ., 0, and, 8.1, software, (, std, OTA, update, ), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh], ]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation, ., Originated, around, upper, stage, oxygen, tank, ., Cause, still, unknown, ., More, soon, .], ]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes, ., This, seems, instant, from, a, human, perspective, ,, but, it, really, a, fast, fire, ,, not, an, explosion, ., Dragon, would, have, been, fine, .], ]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-16 09:30:00-04:00,"[[@AaronS5_, @FrenchieEAP, @karpathy, Yes], [@AustinTeslaClub, @SpaceX, @austinbarnard45, @PPathole, @TeslaOwnersEBay, @bluemoondance74, @teslaownersSV, @JohnnaCrider1, @TeslaNY, Absolutely, !], [@FrenchieEAP, @karpathy, FSD, beta, 9, is, using, the, pure, vision, production, code, for, highway, driving, ., Beta, 10, hopefully, (, Beta, 11, definitely, ), will, use, one, stack, to, rule, them, all, –, city, streets, ,, highway, &, complex, parking, lots, .], [@Erdayastronaut, @SpaceX, Probably, Monday], ]",4,elonmusk,654.679993,650.599976,1.006271,0.627116,TSLA
2021-07-16 16:00:00-04:00,"[[@TeslaNY, Do, you, even, press, ?], [@Teslarati, Improving, permit, approval, speed, &, lowering, permit, costs, for, solar, would, make, a, big, difference], [@RationalEtienne, @etherkragg, Those, are, major, factors], [@enn_nafnlaus, @jpr007, @TesLatino, @klwtts, Indeed, ,, but, again, like, a, parking, lot, ,, a, battery, having, big, “, roads, ”, tends, to, decrease, number, of, “, parking, spaces, ”, (, ie, stores, less, energy, )], ]",4,elonmusk,644.219971,654.679993,0.984023,-1.597731,TSLA
2021-07-19 09:30:00-04:00,"[[@DragTimes, @Tesla, Nice], [@grimnut, @Tesla, @WholeMarsBlog, @DirtyTesla, Haha], [@WholeMarsBlog, You, don, ’, t, even, need, to, touch, the, shifter, in, new, S, ., Auto, detect, direction, will, come, as, an, optional, setting, to, all, cars, with, FSD, .]]",3,elonmusk,629.890015,NaN,NaN,NaN,TSLA


In [ ]:
i = 0
take_from_df = pd.DataFrame()

while i < len(test):
        if test['price'].iloc[i] == 0:
            take_from_df = test.iloc[i]
            j = i + 1
            while test['price'].iloc[j] == 0:
                combined += test.iloc[j]
                j += 1
            username = test.iloc[j]['username']
            price = test.iloc[j]['price']
            percent_change = test.iloc[j]['percent change']
            test.iloc[j] += combined
            test['username'].iloc[j] = username
            test['price'].iloc[j] = price
            test['percent change'].iloc[j] = percent_change
            i = j
        i += 1

In [ ]:
def fix_closed_market_tweets(combined_df):
    combined_df = combined_df.set_index('date')
    i = 0
    combined=pd.DataFrame()

    while i < len(test):
        if test['price'].iloc[i] == 0:
            combined = test.iloc[i]
            j = i + 1
            while test['price'].iloc[j] == 0:
                combined += test.iloc[j]
                j += 1
            username = test.iloc[j]['username']
            price = test.iloc[j]['price']
            percent_change = test.iloc[j]['percent change']
            test.iloc[j] += combined
            test['username'].iloc[j] = username
            test['price'].iloc[j] = price
            test['percent change'].iloc[j] = percent_change
            i = j
        i += 1
    test = test[test['price'] !=0]
    return test

In [13]:
def combine_tweets_stocks_2(ceos_merged, stocks_full):
    new_df = pd.DataFrame()
    full_df = pd.DataFrame()
    for user in ceos_merged['username'].unique():
        ticker = handles_tickers[user]
        username = user
        tweet_df = ceos_merged[ceos_merged['username'] == user]
        stock_df = stocks_full[stocks_full['ticker'] == ticker]
        new_df = new_df.append(tweet_df.merge(stock_df, how='left', on='date')).dropna(subset=['ticker'])  
        new_df = new_df.dropna()
        new_df.reset_index(inplace=True, drop=True)
        new_df = new_df.set_index('date')
        
        i = 0
        combined=pd.DataFrame()

        while i < len(new_df):
            if new_df['price'].iloc[i] == 0:
                combined = new_df.iloc[i]['tweet']
                j = i + 1
                while new_df['price'].iloc[j] == 0:
                    combined.append(new_df.iloc[j]['tweet'])
                    j += 1
                price = new_df.iloc[j]['price']
                stock_return = new_df.iloc[j]['return']
                new_df.iloc[j]['tweet'].append(combined)
                new_df['num_tweets'].iloc[j] = len(new_df['tweet'].iloc[j])
                new_df['price'].iloc[j] = price
                new_df['return'].iloc[j] = stock_return
                i = j
            i += 1
        new_df = new_df[new_df['price'] !=0]
        full_df = full_df.append(new_df)
    return full_df

In [14]:
combined_df_2 = combine_tweets_stocks_2(tweets_combined, stocks_full)

In [25]:
combined_df_2.loc['2016-08-30 09:30:00-04:00']

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-30 09:30:00-04:00,"[[So, sad, Gene, Wilder, has, passed, on--one, of, my, favorite, actors, of, all, time, ., Thank, you, @ConanOBrien, for, this, interview, ., https://t.co/b6ky8uWnbs], ]",1,Benioff,80.000000,79.919998,1.001001,0.100102,CRM
2016-08-30 09:30:00-04:00,"[[That, ’, s, one, way, to, get, to, work, ., But, I, don, ’, t, think, it, would, work, in, #NewYork, or, #Bellevue, ;), https://t.co/nacKTAGo5e], [Lost, one, of, the, world's, greatest, entertainers, today, ..., You, will, be, missed, Gene, Wilder, .], [We, listened, &, we, amped, #TMobileONE, !, !, !, Who's, ready, to, experience, this, 5, days, earlier, ?, !, ?, https://t.co/xaKsItJXQd], [#MondayMotivation, :, #TMobileONE, AMPED, !, !, https://t.co/Ej64Zo0kXY], [ICMYI, :, @ArianaGrande, &, @NickiMinaj, ’, s, EPIC, performance, all, in, #magenta, !, !, !, https://t.co/QLyYcdA5hM], [Ready, for, an, end, of, summer, road, trip, ?, !, ?, Tomorrow, ,, #TMobileTuesdays, +, @Shell, are, helping, you, out, w, /, $, ., 25, off, a, gallon, per, fill, up, !, #GetThanked], [@frwlkr34, @TMobile, We, ’, re, always, listening, to, customers, !, !], [The, #Batmobile, doesn, ’, t, get, great, MPG, ,, but, for, #TMobileTuesdays, @Shell, is, helping, us, save, at, the, pump, !, !, https://t.co/bksBzZO6uW], ]",8,JohnLegere,46.810001,46.730000,1.001712,0.171200,TMUS


In [16]:
def combine_tweets_stocks(ceos_merged, stocks_full):
    new_df = pd.DataFrame()
    for user in ceos_merged['username'].unique():
        ticker = handles_tickers[user]
        tweet_df = ceos_merged[ceos_merged['username'] == user]
        stock_df = stocks_full[stocks_full['ticker'] == ticker]
        new_df = new_df.append(tweet_df.merge(stock_df, how='left', on='date')).dropna(subset=['ticker'])  
    return new_df

def fix_closed_market_tweets(test):
    test = test.dropna()
    test.reset_index(inplace=True, drop=True)
    test = test.set_index('date')
    for i in range(len(test)):
        test['tweet'].iloc[i] += " "

    i = 0
    combined=pd.DataFrame()

    while i < len(test):
        if test['price'].iloc[i] == 0:
            combined = test.iloc[i]
            j = i + 1
            while test['price'].iloc[j] == 0:
                combined += test.iloc[j]
                j += 1
            username = test.iloc[j]['username']
            price = test.iloc[j]['price']
            percent_change = test.iloc[j]['percent change']
            test.iloc[j] += combined
            test['username'].iloc[j] = username
            test['price'].iloc[j] = price
            test['percent change'].iloc[j] = percent_change
            i = j
        i += 1
    test = test[test['price'] !=0]
    return test

In [ ]:
for user in combined_df['username'].unique():
    
    user_df = combined_df[combined_df['username'] == user]
    user_price_null = user_df[user_df['price'].isnull()]
    user_return_not_null = user_df[user_df['return'].notnull()]
    
    new_df = pd.DataFrame()
    
    for user in combined_df['username'].unique():
        user_df = combined_df[combined_df['username'] == user]
        #user_df = user_df.set_index('date')
        user_price_null = user_df[user_df['price'].isnull()]
        user_price_not_null = user_df[user_df['price'].notnull()]
        for i in range(len(user_price_null)):
            first_time_point = user_price_null.iloc[i].name
            next_time_point = user_return_not_null[user_return_not_null.index > first_time_point].iloc[0].name
            temp_df = user_df.loc[first_time_point : next_time_point]
            combined_tweet = []
            combined_user_df = user_price_not_null.copy()
            for j in range(len(temp_df)):
                combined_tweet += temp_df['tweet'].iloc[j]
            print(user, next_time_point)
            combined_user_df.at[next_time_point, 'tweet'] = combined_tweet
            combined_user_df['num_tweets'] = combined_user_df['tweet'].apply(lambda x: len(x))
        new_df = new_df.append(combined_user_df)

In [219]:

new_df = pd.DataFrame()

for user in combined_df['username'].unique():
    user_df = combined_df[combined_df['username'] == user]
    #user_df = user_df.set_index('date')
    user_price_null = user_df[user_df['price'].isnull()]
    user_price_not_null = user_df[user_df['price'].notnull()]
    for i in range(len(user_price_null)):
        first_time_point = user_price_null.iloc[i].name
        next_time_point = user_price_not_null[user_price_not_null.index > first_time_point].iloc[0].name
        temp_df = user_df.loc[first_time_point : next_time_point]
        combined_tweet = []
        for j in range(len(temp_df)):
            combined_tweet += temp_df['tweet'].iloc[j]
        user_price_not_null.at[next_time_point, 'tweet'] = combined_tweet
        user_price_not_null['num_tweets'] = user_price_not_null['tweet'].apply(lambda x: len(x))
    new_df = new_df.append(user_price_not_null)

In [220]:
new_df

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q&amp;A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today]]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress., Maybe, something, to, announce, in, a, few, months., Have, played, all, prior, Deus, Ex., Not, this, one, yet.], [Thanks, for, the, longstanding, faith, in, SpaceX., We, very, much, look, forward, to, doing, this, milestone, flight, with, you., https://t.co/U2UFez0OhY]]",3,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed,, according, to, HK, regulations., Happy, to, do, it, if, regs, change.], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details., Will, publish, on, Tesla, website, later, today.], [Major, improvements, to, Autopilot, coming, with, V8.0, and, 8.1, software, (std, OTA, update), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh]]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation., Originated, around, upper, stage, oxygen, tank., Cause, still, unknown., More, soon.]]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes., This, seems, instant, from, a, human, perspective,, but, it, really, a, fast, fire,, not, an, explosion., Dragon, would, have, been, fine.]]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-16 16:00:00-04:00,"[[BUT, WHERE, IS, THE, SOLID, PINK, HEART?!!, https://t.co/wF8LWtG6dx], [Trust, science!, It’s, important, to, continue, staying, safe,, the, COVID-19, pandemic, isn’t, over, yet!, https://t.co/HBhDZCbvrk]]",2,JohnLegere,149.410004,148.089996,1.008914,0.891355,TMUS
2021-07-19 09:30:00-04:00,"[[is, it, possible, that, the, ocean, makes, the, morning, coffee, ☕️, even, better, than, it's, normal, great?, 😊, https://t.co/43jhIroufd], [Hey, New, Yorkers, –, need, an, officiant?!, https://t.co/PbBecmjgu0], [Who’s, subscribing??, https://t.co/bZr4a84P2X], [Number, 16, is, a, movie, night, game, changer!!, https://t.co/9YJS0H383P]]",4,JohnLegere,147.500000,NaN,NaN,NaN,TMUS
2021-07-19 16:00:00-04:00,"[[What, I’ve, learned, from, @Lifehacker, is, that, I, do, EVERYTHING, wrong!, https://t.co/xAbzXxZhqZ], [Always, love, a, good, Quesarito, https://t.co/KaiMmXEc5q], [So, I'm, about, to, go, for, a, run, on, the, beach.....am, i, the, only, one, that, doesn't, think, these, ☁️, mean, 🌧, ☔️, is, coming?, https://t.co/fMxzuOZ0nF], [Gorgeous!, Maybe, I, should, charter, a, magenta, rocket., https://t.co/jx2ZL0gX6Q]]",4,JohnLegere,144.610001,147.500000,0.980407,-1.959322,TMUS


In [17]:
fix_closed_market_tweets(combined_df)

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q, &, A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today], ]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX, ., We, very, much, look, forward, to, doing, this, milestone, flight, with, you, ., https://t.co/U2UFez0OhY], ]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed, ,, according, to, HK, regulations, ., Happy, to, do, it, if, regs, change, .], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details, ., Will, publish, on, Tesla, website, later, today, .], [Major, improvements, to, Autopilot, coming, with, V8, ., 0, and, 8.1, software, (, std, OTA, update, ), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh], ]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation, ., Originated, around, upper, stage, oxygen, tank, ., Cause, still, unknown, ., More, soon, .], ]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes, ., This, seems, instant, from, a, human, perspective, ,, but, it, really, a, fast, fire, ,, not, an, explosion, ., Dragon, would, have, been, fine, .], ]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-16 09:30:00-04:00,"[[More, additions, to, Clubhouse, !, Who, still, actively, uses, ?, ?, !, https://t.co/OCwsKhscIv], [Congratulations, to, @neilbarua, and, the, @ServiceMax, team, !, !, !, https://t.co/AFaIqM1I48], ]",2,JohnLegere,148.089996,148.339996,0.998315,-0.168532,TMUS
2021-07-16 16:00:00-04:00,"[[BUT, WHERE, IS, THE, SOLID, PINK, HEART, ?, !, !, https://t.co/wF8LWtG6dx], [Trust, science, !, It, ’, s, important, to, continue, staying, safe, ,, the, COVID, -, 19, pandemic, isn, ’, t, over, yet, !, https://t.co/HBhDZCbvrk], ]",2,JohnLegere,149.410004,148.089996,1.008914,0.891355,TMUS
2021-07-19 16:00:00-04:00,"[[What, I, ’, ve, learned, from, @Lifehacker, is, that, I, do, EVERYTHING, wrong, !, https://t.co/xAbzXxZhqZ], [Always, love, a, good, Quesarito, https://t.co/KaiMmXEc5q], [So, I'm, about, to, go, for, a, run, on, the, beach, ..., am, i, the, only, one, that, doesn't, think, these, ☁, ️, mean, 🌧, ☔, ️, is, coming, ?, https://t.co/fMxzuOZ0nF], [Gorgeous, !, Maybe, I, should, charter, a, magenta, rocket, ., https://t.co/jx2ZL0gX6Q], ]",4,JohnLegere,144.610001,147.500000,0.980407,-1.959322,TMUS


In [18]:
combined_df

,date,tweet,num_tweets,username,price,shifted,change,return,ticker
22,2016-08-23 16:00:00-04:00,"[[Journalist, Q, &, A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today], ]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
23,2016-08-28 09:30:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress, ., Maybe, something, to, announce, in, a, few, months, ., Have, played, all, prior, Deus, Ex, ., Not, this, one, yet, .]]",2,elonmusk,NaN,NaN,NaN,NaN,TSLA
24,2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX, ., We, very, much, look, forward, to, doing, this, milestone, flight, with, you, ., https://t.co/U2UFez0OhY], ]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
25,2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed, ,, according, to, HK, regulations, ., Happy, to, do, it, if, regs, change, .], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details, ., Will, publish, on, Tesla, website, later, today, .], [Major, improvements, to, Autopilot, coming, with, V8, ., 0, and, 8.1, software, (, std, OTA, update, ), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh], ]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
26,2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation, ., Originated, around, upper, stage, oxygen, tank, ., Cause, still, unknown, ., More, soon, .], ]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
...,...,...,...,...,...,...,...,...,...
3412,2021-07-18 16:00:00-04:00,"[[is, it, possible, that, the, ocean, makes, the, morning, coffee, ☕, ️, even, better, than, it's, normal, great, ?, 😊, https://t.co/43jhIroufd], [Hey, New, Yorkers, –, need, an, officiant, ?, !, https://t.co/PbBecmjgu0], [Who, ’, s, subscribing, ?, ?, https://t.co/bZr4a84P2X]]",3,JohnLegere,NaN,NaN,NaN,NaN,TMUS
3413,2021-07-19 09:30:00-04:00,"[[Number, 16, is, a, movie, night, game, changer, !, !, https://t.co/9YJS0H383P]]",1,JohnLegere,147.500000,NaN,NaN,NaN,TMUS
3414,2021-07-19 16:00:00-04:00,"[[What, I, ’, ve, learned, from, @Lifehacker, is, that, I, do, EVERYTHING, wrong, !, https://t.co/xAbzXxZhqZ], [Always, love, a, good, Quesarito, https://t.co/KaiMmXEc5q], [So, I'm, about, to, go, for, a, run, on, the, beach, ..., am, i, the, only, one, that, doesn't, think, these, ☁, ️, mean, 🌧, ☔, ️, is, coming, ?, https://t.co/fMxzuOZ0nF], [Gorgeous, !, Maybe, I, should, charter, a, magenta, rocket, ., https://t.co/jx2ZL0gX6Q], ]",4,JohnLegere,144.610001,147.500000,0.980407,-1.959322,TMUS
3415,2021-07-20 09:30:00-04:00,"[[Call, on, John, is, POSTPONED, for, today, but, tune, in, on, Thursday, ,, July, 22nd, at, 12, PM, ET, on, Instagram, Stories, for, an, extra, special, Call, on, John, !, What, do, you, want, to, see, me, give, away, ?, ?, https://t.co/4P3hucQ3io], ]",1,JohnLegere,144.789993,144.610001,1.001245,0.124468,TMUS


In [19]:
def fix_closed_market_tweets(df):
    # set date index
    df = df.dropna()
    df.reset_index(inplace=True, drop=True)
    df = df.set_index('date')
    
    combined=pd.DataFrame()
    i = 0
    while i < len(df):
        if df['price'].iloc[i] == 0:
            combined = df.iloc[i]
            j = i + 1
            while df['price'].iloc[j] == 0:
                combined += df.iloc[j]
                j += 1
            username = df.iloc[j]['username']
            price = df.iloc[j]['price']
            percent_change = df.iloc[j]['percent change']
            df.iloc[j] += combined
            df['username'].iloc[j] = username
            df['price'].iloc[j] = price
            df['percent change'].iloc[j] = percent_change
            i = j
        i += 1
    df = df[df['price'] !=0]
    return df

In [20]:
test = combined_df[combined_df['username'] == 'elonmusk']

In [21]:
test

,date,tweet,num_tweets,username,price,shifted,change,return,ticker
22,2016-08-23 16:00:00-04:00,"[[Journalist, Q, &, A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today], ]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
23,2016-08-28 09:30:00-04:00,"[[@Kotaku, one, of, my, favorite, games, as, a, kid], [@BelovedRevol, Making, progress, ., Maybe, something, to, announce, in, a, few, months, ., Have, played, all, prior, Deus, Ex, ., Not, this, one, yet, .]]",2,elonmusk,NaN,NaN,NaN,NaN,TSLA
24,2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX, ., We, very, much, look, forward, to, doing, this, milestone, flight, with, you, ., https://t.co/U2UFez0OhY], ]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
25,2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed, ,, according, to, HK, regulations, ., Happy, to, do, it, if, regs, change, .], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details, ., Will, publish, on, Tesla, website, later, today, .], [Major, improvements, to, Autopilot, coming, with, V8, ., 0, and, 8.1, software, (, std, OTA, update, ), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh], ]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
26,2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation, ., Originated, around, upper, stage, oxygen, tank, ., Cause, still, unknown, ., More, soon, .], ]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
...,...,...,...,...,...,...,...,...,...
2194,2021-07-18 09:30:00-04:00,"[[@ArtifactsHub, And, all-time, hodl, champion], [@ArtifactsHub, OG, Hipster], [@ValaAfshar, Indeed], [@waitbutwhy, Pohtaytohz], [@squawksquare, Current, Summon, is, sometimes, useful, ,, but, mostly, just, a, fun, trick, ., Once, we, move, summon, (, plus, highway, driving, ), to, a, single, FSD, stack, ,, it, will, be, sublime, .], [Cybrrrtruck, https://t.co/rdiMFdYOS6]]",6,elonmusk,NaN,NaN,NaN,NaN,TSLA
2195,2021-07-18 16:00:00-04:00,"[[@thePiggsBoson, Problem, 1st, ,, theory, 2nd, is, for, sure, way, to, go, ,, as, it, establishes, relevance, ,, thus, improving, memory, retention]]",1,elonmusk,NaN,NaN,NaN,NaN,TSLA
2196,2021-07-19 09:30:00-04:00,"[[@DragTimes, @Tesla, Nice], [@grimnut, @Tesla, @WholeMarsBlog, @DirtyTesla, Haha], [@WholeMarsBlog, You, don, ’, t, even, need, to, touch, the, shifter, in, new, S, ., Auto, detect, direction, will, come, as, an, optional, setting, to, all, cars, with, FSD, .]]",3,elonmusk,629.890015,NaN,NaN,NaN,TSLA
2197,2021-07-19 16:00:00-04:00,"[[@jack, @BitcoinMagazine, @CathieDWood, Sure, ,, I, have, a, ton], [@BitcoinMagazine, @jack, @CathieDWood, During, this, talk, ,, we, will, sing, a, cover, of, The, Final, Countdown, by, Europe, https://t.co/7YUXiW8dhd], ]",2,elonmusk,646.219971,629.890015,1.025925,2.592509,TSLA


In [22]:
fix_closed_market_tweets(test)

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[[Journalist, Q, &, A, for, 30, mins, and, embargo, ends, at, 12:30], [Tesla, product, announcement, at, noon, California, time, today], ]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[[Thanks, for, the, longstanding, faith, in, SpaceX, ., We, very, much, look, forward, to, doing, this, milestone, flight, with, you, ., https://t.co/U2UFez0OhY], ]",1,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[[@Lockyep, Not, allowed, ,, according, to, HK, regulations, ., Happy, to, do, it, if, regs, change, .], [We, need, to, do, one, more, minor, rev, on, 8.0, and, then, will, go, to, wide, release, in, a, few, weeks], [Writing, post, now, with, details, ., Will, publish, on, Tesla, website, later, today, .], [Major, improvements, to, Autopilot, coming, with, V8, ., 0, and, 8.1, software, (, std, OTA, update, ), primarily, through, advanced, processing, of, radar, signals], [@newscientist, uh, oh], ]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,"[[Loss, of, Falcon, vehicle, today, during, propellant, fill, operation, ., Originated, around, upper, stage, oxygen, tank, ., Cause, still, unknown, ., More, soon, .], ]",1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[[Finishing, Autopilot, blog, postponed, to, end, of, weekend], [@scrappydog, yes, ., This, seems, instant, from, a, human, perspective, ,, but, it, really, a, fast, fire, ,, not, an, explosion, ., Dragon, would, have, been, fine, .], ]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA
...,...,...,...,...,...,...,...,...
2021-07-15 16:00:00-04:00,"[[@Erdayastronaut, @Model3Owners, How, about, a, wifi, camera, link, ?], [@Model3Owners, To, be, frank, ,, there, is, always, some, chance, that, Cybertruck, will, flop, ,, because, it, is, so, unlike, anything, else, ., I, don, ’, t, care, ., I, love, it, so, much, even, if, others, don, ’, t, ., Other, trucks, look, like, copies, of, the, same, thing, ,, but, Cybertruck, looks, like, it, was, made, by, aliens, from, the, future, .], [@johnkrausphotos, @SpaceX, @PortCanaveral, Version, 3, of, the, SpaceX, droneship, ., Team, did, great, work, !, Will, be, epic, to, see, the, deep, sea, oil, rigs, converted, to, ocean, spaceports, for, Starship, .], [@BLKMDL3, @Model3Owners, In, end, ,, we, kept, production, design, almost, exactly, same, as, show, car, ., Just, some, small, tweaks, here, &, there, to, make, it, slightly, better, ., No, door, handles, ., Car, recognizes, you, &, opens, door, ., Having, all, four, wheels, steer, is, amazing, for, nimble, handling, &, tight, turns, !], [@TesLatino, @klwtts, @jpr007, Tapering, down, charge, rate, is, simply, a, physical, thing, that, has, to, happen, ,, as, lithium, ions, bounce, around, what, is, an, increasingly, full, “, parking, lot, ”, ., Just, like, a, car, parking, lot, ,, where, it, takes, longer, to, find, a, spot, when, the, lot, is, almost, full, .], ]",5,elonmusk,650.599976,658.390015,0.988168,-1.183195,TSLA
2021-07-16 09:30:00-04:00,"[[@AaronS5_, @FrenchieEAP, @karpathy, Yes], [@AustinTeslaClub, @SpaceX, @austinbarnard45, @PPathole, @TeslaOwnersEBay, @bluemoondance74, @teslaownersSV, @JohnnaCrider1, @TeslaNY, Absolutely, !], [@FrenchieEAP, @karpathy, FSD, beta, 9, is, using, the, pure, vision, production, code, for, highway, driving, ., Beta, 10, hopefully, (, Beta, 11, definitely, ), will, use, one, stack, to, rule, them, all, –, city, streets, ,, highway, &, complex, parking, lots, .], [@Erdayastronaut, @SpaceX, Probably, Monday], ]",4,elonmusk,654.679993,650.599976,1.006271,0.627116,TSLA
2021-07-16 16:00:00-04:00,"[[@TeslaNY, Do, you, even, press, ?], [@Teslarati, Improving, permit, approval, speed, &, lowering, permit, costs, for, solar, would, make, a, big, difference], [@RationalEtienne, @etherkragg, Those, are, major, factors], [

In [23]:
def fix_closed_market_tweets(test):
    test = test.dropna()
    test.reset_index(inplace=True, drop=True)
    test = test.set_index('date')
    for i in range(len(test)):
        test['tweet'].iloc[i] += " "

    i = 0
    combined=pd.DataFrame()

    while i < len(test):
        if test['price'].iloc[i] == 0:
            combined = test.iloc[i]
            j = i + 1
            while test['price'].iloc[j] == 0:
                combined += test.iloc[j]
                j += 1
            username = test.iloc[j]['username']
            price = test.iloc[j]['price']
            percent_change = test.iloc[j]['percent change']
            test.iloc[j] += combined
            test['username'].iloc[j] = username
            test['price'].iloc[j] = price
            test['percent change'].iloc[j] = percent_change
            i = j
        i += 1
    test = test[test['price'] !=0]
    return test